In [150]:
import requests
from bs4 import BeautifulSoup
import bs4.element
import datetime
import warnings
warnings.filterwarnings('ignore')
import re
import pandas as pd
import time
from datetime import datetime

In [151]:
import pymysql
conn = pymysql.connect(
    host='34.64.240.96'
    , user='root'
    , password='tndusWkd1.'
    , db='final_project'
    , charset='utf8'
)
cur = conn.cursor()

In [152]:
news_df = pd.DataFrame(
    data=None,
    index=None,
    columns=['date','title','link','content']
)

#중복 행 지우기
news_df = news_df.drop_duplicates(keep='first',ignore_index=True)
print("중복 제거 후 행 개수: ",len(news_df))

중복 제거 후 행 개수:  0


In [153]:
# ConnectionError방지
headers = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36'}

# BeautifulSoup 객체 생성
def get_soup_obj(url):
    res = requests.get(url, headers = headers, verify=False)
    soup = BeautifulSoup(res.text,'lxml')
    
    return soup

In [154]:
# 뉴스의 기본 정보 가져오기
def get_top3_news_info():
    news_urls =[]
    
    # 해당 분야 상위 뉴스 목록 주소
    sec_url = "https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1=101"
    
    # 해당 분야 상위 뉴스 HTML 가져오기
    soup = get_soup_obj(sec_url)
  
    # 해당 분야 상위 뉴스 3개 가져오기

    lis3 = soup.find('ul', class_='type06_headline').find_all("li", limit=3)
    for li in lis3:
        news_url = li.a.attrs.get('href')
        news_urls.append(news_url)

    
    print(news_urls)
    return news_urls

In [155]:
# 상위 3개 뉴스 크롤링
def F_crawling(news_urls):
    news_titles=[]
    news_contents=[]
    news_dates=[]

    for url in news_urls:
        news_html = get_soup_obj(url)


        # 뉴스 제목 가져오기
        title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
        if title == None:
            title = news_html.select_one("#content > div.end_ct > div > h2")

 
        # 뉴스 본문 가져오기
        content = news_html.select("div#dic_area")
        if content == []:
            content = news_html.select("#articeBody")
            
        # 기사 텍스트만 가져오기
        # list합치기
        content = ''.join(str(content))

        # html태그제거 및 텍스트 다듬기
        pattern1 = '<[^>]*>'
        title = re.sub(pattern=pattern1, repl='', string=str(title))
        content = re.sub(pattern=pattern1, repl='', string=content)
        pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
        content = content.replace(pattern2, '')

        news_titles.append(title)
        news_contents.append(content)

        html_date = news_html.select_one("div#ct > div.media_end_head > div.media_end_head_info > div span")
        news_date = html_date.attrs['data-date-time']
        news_dates.append(news_date)

    a = pd.DataFrame({'date':news_dates, 'title':news_titles, 'link':news_urls, 'content':news_contents})

    #중복 행 지우기
    a = a.drop_duplicates(keep='first',ignore_index=True)

    return a

In [156]:
# def clearnews():
#     special_char = ' \/:*?\"\"<>|\'\' '
#     for clear in special_char:
#         if clear in title:
#             print(title.find(clear),clear)
#             title = title.replace(clear,'')
#     print(title)

# def clearnews():
#     special_char = ' \/:*?\"\"<>|\'\' '
#     for clear in special_char:
#         if clear in content:
#             print(content.find(clear),clear)
#             content = content.replace(clear,'')
#     print(content)

def text_clean(text):
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' # E-mail제거
    text = re.sub(pattern, '', text)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+' # URL제거
    text = re.sub(pattern, '', text)
    pattern = '[a-zA-Z0-9]'    # 숫자와 알파벳 제거
    text = re.sub(pattern, '', text)
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
    text = re.sub(pattern, '', text)
    pattern = '<[^>]*>'         # HTML 태그 제거
    text = re.sub(pattern, '', text)
    pattern = '[^\w\s]'         # 특수기호제거
    text = re.sub(pattern, '', text)
    pattern = '[\n]'            # \n제거
    text = re.sub(pattern, '', text)
    pattern = '[\t]'            # \n제거
    text = re.sub(pattern, '', text)
    return text  

In [157]:
# while True:
news_urls = get_top3_news_info()
a = F_crawling(news_urls)
news_df=pd.concat([news_df,a])
news_df = news_df.drop_duplicates(keep='first',ignore_index=True)
print(news_df.head())

# time.sleep(50)

['https://n.news.naver.com/mnews/article/003/0011958077?sid=101', 'https://n.news.naver.com/mnews/article/032/0003234469?sid=101', 'https://n.news.naver.com/mnews/article/003/0011958075?sid=101']
                  date                              title   
0  2023-07-06 15:26:12  "그때 그시절" 17년전 산돌 폰트 만난다…'SD픽셀' 출시  \
1  2023-07-06 15:26:08          예적금 보호 안내문 붙은 새마을금고[경향포토]   
2  2023-07-06 15:26:06     NH농협은행, 개인사업자 저금리 '지자체협약대출' 출시   

                                                link   
0  https://n.news.naver.com/mnews/article/003/001...  \
1  https://n.news.naver.com/mnews/article/032/000...   
2  https://n.news.naver.com/mnews/article/003/001...   

                                             content  
0  [\n당시 복원해 픽셀 폰트로 재탄생세기말 감성의 손글씨 느낌 10종\n\n\n\n...  
1  [\n\n\n\n\n새마을금고 건전성에 대한 우려로 정부가 '범정부 대응단'을 구성...  
2  [\n\n\n\n\n[서울=뉴시스]이주혜 기자 = NH농협은행은 지방자치단체와 협약...  


In [158]:
NewsDate = news_df['date'].to_list()
NewsTItle = news_df['title'].to_list()
NewsUrl = news_df['link'].to_list()
NewsContent = news_df['content'].to_list()

In [159]:
for date,title,url,content in zip(NewsDate, NewsTItle, NewsUrl, NewsContent):
    sql = "INSERT INTO real_news (NewsDates,NewsTitles,NewsUrls,NewsContents) VALUES (%s, %s, %s, %s)" % ("'"+date+"'", "'"+title+"'", "'"+url+"'", "'"+content+"'")
    print(sql)
    cur.execute(sql)
conn.commit()
conn.close()

INSERT INTO real_news (NewsDates,NewsTitles,NewsUrls,NewsContents) VALUES ('2023-07-06 15:26:12', '"그때 그시절" 17년전 산돌 폰트 만난다…'SD픽셀' 출시', 'https://n.news.naver.com/mnews/article/003/0011958077?sid=101', '[
당시 복원해 픽셀 폰트로 재탄생세기말 감성의 손글씨 느낌 10종



[서울=뉴시스] 산돌 'SD 픽셀' 시리즈. (이미지=산돌 제공) 2023.07.06. photo@newsis.com[서울=뉴시스] 배민욱 기자 = 크리에이터 콘텐츠 플랫폼 산돌은 17년 전 인기를 끌었던 자사 픽셀 폰트를 재해석해 현대화한 'SD 픽셀' 시리즈 10종을 선보인다고 6일 밝혔다.   SD 픽셀 시리즈는 2000년대 초반 웹폰트로 처음 공개됐다. 2006년 싸이월드 등에서 폰트 판매 서비스를 시작하는 동시에 자신의 개성을 표현할 수 있는 수단으로 이용됐다. 산돌은 픽셀 폰트 시리즈를 최근 인기인 Y2K 트렌드에 맞게 복원하고 가공해 새롭게 선보였다. SD 픽셀쪽지,  SD 픽셀비밀쪽지, SD 픽셀굿모닝, SD 픽셀굿밤, SD 픽셀동동동, SD 픽셀쁘띠공주, SD 픽셀사과나무, SD픽셀섬섬, SD픽셀요술지팡이, SD픽셀종이학까지 세기말 감성이 담긴 귀여운 손글씨 느낌의 폰트가 총 10종으로 출시됐다. 특히 SD 픽셀 시리즈 중 SD 픽셀쪽지, SD 픽셀비밀쪽지는 모바일에서 사용할 수 있도록 선보였다. iOS 사용자의 경우 산돌구름 모바일 앱에서, 안드로이드 사용자의 경우 갤럭시 스토어를 통해 구매할 수 있다. 사용자는 카카오톡 등의 메신저, 그 외 모바일 서비스에서 폰트를 적용해 이용이 가능하다. 산돌은 SD 픽셀 시리즈 출시를 맞아 세기말 감성의 디지털 굿즈 '카카오톡 테마'와 'Y2K 인스타그램 필터'를 함께 제작해 무료로 제공한다. 카카오톡 테마는 산돌구름의 블로그에서 다운로드 받아 사용할 수 있다. 인스타그램 필터는

ProgrammingError: (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near 'SD픽셀' 출시', 'https://n.news.naver.com/mnews/article/003/0011958077?sid...' at line 1")

In [ ]:
# ###데이터 프레임으로 만들기###
# from datetime import date, timedelta
# import pandas as pd

# #데이터 프레임 만들기

# # list=[]
# a = pd.DataFrame({'date':news_dates,'title':news_titles,'link':news_urls,'content':news_contents})

# # list.append([news_df,a])

# #데이터 프레임 저장
# now = datetime.now() 
# a.to_csv('C:\\big16\\trading_project\\ssuyan\\REAL_NEWS\\_{}.csv'.format(now.strftime('%Y%m%d_%H%M%S')),encoding='utf-8-sig',index=False)
